In [100]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import re
import time

In [203]:
driver = webdriver.Chrome(r'C:\Users\Chun\chromedriver_win32 (1)\chromedriver.exe')
driver.get("https://www.bluenile.com/build-your-own-ring/settings")

SCROLL_PAUSE_TIME = 0.5


while True:

    # Get scroll height
    ### This is the difference. Moving this *inside* the loop
    ### means that it checks if scrollTo is still scrolling 
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:

        # try again (can be removed)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        # check if the page height has remained the same
        if new_height == last_height:
            # if so, you are done
            break
        # if not, move on to the next loop
        else:
            last_height = new_height
            continue

In [30]:
links = driver.find_elements_by_class_name('related-metals')

link_list = []
for link in links:
    item_link = link.get_attribute("href")
    link_list.append(item_link)

In [31]:
# for link in link_list:
#     link.click()

link1 = link_list[0]
driver.get(link1)

### 1.style

In [32]:
style = driver.find_element_by_class_name('title').text
print(style)

Petite Twist Diamond Engagement Ring


### 2.metal
### 3.side stone weight

In [33]:
band = driver.find_element_by_class_name('subtitle').text
print(band)

14k White Gold (1/10 ct. tw.)


In [167]:
s = re.search('(.+)\((.*)ct. tw.\)',band)
try:
    weight = s.group(2)
    a,b = re.search('(\d)\/(\d+)', weight).groups()
    side_weight = int(a)/int(b)
except:
    side_weight = 0
try:    
    metal = s.group(1)
except: metal = band
print(metal)
print(side_weight)

Platinum
0


### 4. price

In [184]:
# price = driver.find_element_by_class_name('price-label').text
# price = int(price.replace('$','').replace(',',''))
# price
for i in range(0):
    print(i)

### 5.avearge rating

In [199]:
driver.get('https://www.bluenile.com/build-your-own-ring/scalloped-pave-diamond-engagement-ring-platinum_33601')
# time.sleep(1)
# avg_rating = driver.find_element_by_xpath('//div[@class="bottom-line-items"]/span[@class="avg-score font-color-gray-darker"]').text
# avg_rating = float(avg_rating)
# avg_rating

try:
    time.sleep(2)
    avg_rating = driver.find_element_by_xpath('//div[@class="bottom-line-items"]/span[@class="avg-score font-color-gray-darker"]').text
    avg_rating = float(avg_rating)
except:
    avg_rating = None
    
print(avg_rating)

4.9


### 6.numer of reviews

In [181]:
driver.get('https://www.bluenile.com/build-your-own-ring/split-shank-plain-pave-diamond-engagement-ring-14k-rose-gold_77808')
num_review = driver.find_element_by_class_name('rating-values').text
num_review = int(re.search('(\d+) reviews?',num_review).group(1))
num_review

'1 review'

In [208]:
driver.get('https://www.bluenile.com/build-your-own-ring/petite-twist-diamond-engagement-ring-14k-white-gold_56003?yoReviewsPage=2')
reviews = driver.find_elements_by_xpath('//div[@class="yotpo-reviews yotpo-active"]/div')
reviews = reviews[3:]
len(reviews)

17

## retrive all the reviews

### go through all review pages

In [212]:
import time

driver = webdriver.Chrome(r'C:\Users\Chun\chromedriver_win32 (1)\chromedriver.exe')
driver.get("https://www.bluenile.com/build-your-own-ring/settings")

PAUSE_TIME = 2


while True:

    # Get scroll height
    ### This is the difference. Moving this *inside* the loop
    ### means that it checks if scrollTo is still scrolling 
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:

        # try again (can be removed)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        # check if the page height has remained the same
        if new_height == last_height:
            # if so, you are done
            break
        # if not, move on to the next loop
        else:
            last_height = new_height
            continue
            
## get all the links for each ring style ##
links = driver.find_elements_by_class_name('related-metals')

product_urls = []
for link in links:
    item_link = link.get_attribute("href")
    product_urls.append(item_link)
    
    
## Make csv_writer ##
csv_file = open('bluenile.csv', 'w', encoding='utf-8', newline='')
writer = csv.writer(csv_file)

writer.writerow(['product_name', 'avg_rating', 'side_stone_weight', 'metal',
                 'user_name', 'verified', 'stars', 'date', 'location',
                 'expert_consulted', 'recommended', 'why_buy'])


## enter each product page, retrive data ##
for product_url in product_urls:
    item = {}
    driver.get(product_url)
    time.sleep(PAUSE_TIME)
    
    ## find out how many page of review there are ##
    try:
        num_review = driver.find_element_by_class_name('rating-values').text
        num_review = int(re.search('(\d+) reviews?',num_review).group(1))
        review_page_num = math.ceil(num_review/15)
        print(num_review)
    except:
        num_review = None
    
    if num_review:
        review_page_urls = [f'{product_url}?yoReviewsPage={i+1}' for i in range(review_page_num)]
    else:
        review_page_urls = [f'{product_url}']
    
    ## retrive product information ##
    product_name = driver.find_element_by_class_name('title').text
    
    try:
        time.sleep(PAUSE_TIME)
        avg_rating = driver.find_element_by_xpath('//div[@class="bottom-line-items"]/span[@class="avg-score font-color-gray-darker"]').text
        avg_rating = float(avg_rating)
    except:
        avg_rating = None
    
    band = driver.find_element_by_class_name('subtitle').text
    s = re.search('(.+)\((.*)ct. tw.\)',band)
    try:
        weight = s.group(2)
        a,b = re.search('(\d)\/(\d+)', weight).groups()
        side_weight = int(a)/int(b)
    except:
        side_weight = 0
    try:    
        metal = s.group(1)
    except: 
        metal = band
    
    ## collect review information ##
    if num_review:
        for review_page in review_page_urls:
            driver.get(review_page)
            time.sleep(PAUSE_TIME)
        
            reviews = driver.find_elements_by_xpath('//div[@class="yotpo-reviews yotpo-active"]/div')
            reviews = reviews[3:]
            print(len(reviews))

            for review in reviews:
                question_list = review.find_elements_by_class_name('yotpo-user-field')
            
                row_dict = {}
                for question_ans in question_list:
                    key = question_ans.find_element_by_xpath('./span[1]').text
                    value = question_ans.find_element_by_xpath('./span[2]').text
                    if not key:
                        continue

                    row_dict[key.replace(':', '')] = value

                try:
                    user_name = review.find_element_by_xpath('.//span[@class="y-label yotpo-user-name yotpo-font-bold pull-left"]').text
                except:
                    continue
                try:
                    verified = review.find_element_by_xpath('.//span[@class="y-label yotpo-user-title yotpo-action-hover"]').text
                except:
                    verified = 'Non'
                stars = review.find_element_by_xpath('.//span[@aria-label="Link to product page"]').text
                stars = int(stars[:1])
                date = review.find_element_by_xpath('.//span[@class="y-label yotpo-review-date"]').text
                review_text = review.find_element_by_xpath

                item['product_name'] = product_name
                item['avg_rating'] = avg_rating
                item['side_stone_weight'] = side_weight
                item['metal'] = metal
                item['user_name'] = user_name
                item['verified'] = verified
                item['stars'] = stars
                item['date'] = date
                item['location'] = row_dict.get('Location')
                item['expert_consulted'] = row_dict.get('Blue Nile jewelry expert consulted')
                if row_dict.get('Recommendation'):
                    item['recommended'] = row_dict.get('Recommendation')
                else:
                    item['recommended'] = row_dict.get('Recommend')
                if row_dict.get('Why buy a Blue Nile Diamond'):
                    item['why_buy'] = row_dict.get('Why buy a Blue Nile Diamond')
                else:
                    item['why_buy'] = row_dict.get('Reason')
                
                writer.writerow(item.values())

    else:
        driver.get(review_page_urls[0])
        
        item['product_name'] = product_name
        item['avg_rating'] = avg_rating
        item['side_stone_weight'] = side_weight
        item['metal'] = metal
        item['user_name'] = None
        item['verified'] = None
        item['stars'] = None
        item['date'] = None
        item['location'] = None
        item['expert_consulted'] = None
        item['recommended'] = None
        item['why_buy'] = None

        writer.writerow(item.values())
    

156
18
17
19
17
20
18
16
16
16
16
7
151
17
18
20
22
18
20
17
16
16
16
2
90
21
19
18
17
16
16
110
22
19
16
19
17
16
16
6
965
20
19
21
16
21
17
21
19
19
19
17
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
6
90
18
20
18
20
17
15
78
17
18
17
16
16
4
252
18
19
18
20
17
22
19
18
16
16
16
16
16
16
16
16
13
62
23
16
16
16
3
111
19
18
17
16
16
16
16
6
166
18
18
19
16
16
16
16
16
16
16
16
2
174
19
20
19
20
17
18
17
19
16
16
16
9
151
20
18
16
16
16
16
16
16
16
16
2
126
21
17
16
16
16
16
16
16
7
91
19
19
16
16
16
16
2
72
17
18
16
16
13
66
21
19
22
21
7
171
17
17
16
16
16
16
16
16
16
16
16
7
51
17
17
16
7
100
19
21
16
16
16
16
11
17
18
3
339
18
19
20
20
21
17
17
20
16
17
16
16
16
16
16
16
16
16
16
16
16
16
10
29
18
17
22
18
8
187
24
19
18
21
17
16
16
16
16
16
16
16
8
112
21
17
18
16
16
16
16
8
15
16
104
20
18
16
16
16
16
15
2
2
28
19
14
31
19
17
2
54
20
19
20
10
50
17
16
16
6
3
4
369
23


In [89]:
import math
review_page_num = math.ceil(num_review/15)
review_page_num

11

In [109]:
reviews = driver.find_elements_by_xpath('//div[@class="yotpo-reviews yotpo-active"]/div')
reviews = reviews[3:]
review = reviews[0]
review.

In [52]:
next_button = driver.find_element_by_xpath('//div/a[@rel="next"]')
link_to_next_review = next_button.get_attribute('href')
driver.get(link_to_next_review)

In [92]:
url_list = [f'https://www.bluenile.com/build-your-own-ring/petite-twist-diamond-engagement-ring-14k-white-gold_56003?yoReviewsPage={i+1}' 
            for i in range(review_page_num)]

In [93]:
url_list

['https://www.bluenile.com/build-your-own-ring/petite-twist-diamond-engagement-ring-14k-white-gold_56003?yoReviewsPage=1',
 'https://www.bluenile.com/build-your-own-ring/petite-twist-diamond-engagement-ring-14k-white-gold_56003?yoReviewsPage=2',
 'https://www.bluenile.com/build-your-own-ring/petite-twist-diamond-engagement-ring-14k-white-gold_56003?yoReviewsPage=3',
 'https://www.bluenile.com/build-your-own-ring/petite-twist-diamond-engagement-ring-14k-white-gold_56003?yoReviewsPage=4',
 'https://www.bluenile.com/build-your-own-ring/petite-twist-diamond-engagement-ring-14k-white-gold_56003?yoReviewsPage=5',
 'https://www.bluenile.com/build-your-own-ring/petite-twist-diamond-engagement-ring-14k-white-gold_56003?yoReviewsPage=6',
 'https://www.bluenile.com/build-your-own-ring/petite-twist-diamond-engagement-ring-14k-white-gold_56003?yoReviewsPage=7',
 'https://www.bluenile.com/build-your-own-ring/petite-twist-diamond-engagement-ring-14k-white-gold_56003?yoReviewsPage=8',
 'https://www.bl

In [95]:
for idx, url in enumerate(url_list, 1):
#     next_button = driver.find_element_by_xpath('//div/a[@rel="next"]')
#     next_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//div/a[@rel="next"]')))
#     link_to_next_review = next_button.get_attribute('href')
#     print(link_to_next_review)
#     driver.get(link_to_next_review)
    driver.get(url)
    print(f'coming to page {idx}')
# WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,'//div/a[@rel="next"]')))

coming to page 1
coming to page 2
coming to page 3
coming to page 4


KeyboardInterrupt: 

### 7.customer names

In [111]:
# driver.get('https://www.bluenile.com/build-your-own-ring/petite-twist-diamond-engagement-ring-14k-white-gold_56003')
# reviews = driver.find_elements_by_xpath('//div[@class="yotpo-header-element "]/div')
customer_list = driver.find_elements_by_xpath('//span[@class="y-label yotpo-user-name yotpo-font-bold pull-left"]')
customers = {}
for c in customer_list[1:]:
    customer = c.text
    print(customer)

Noah S.
Ray W.
JONATHAN T.
Zack C.
tim p.
Emmanuel P.
Curtis M.
Sherman E.
Darroch M.
Stefan T.
Colin S.
William F.
Kevin S.
Roger N.
Frank R.


### 8. review date

In [13]:
date = driver.find_elements_by_xpath('//span[@class="y-label yotpo-review-date"]')
for i,d in enumerate(date[2:]):
    d_ = d.text
    if i%2 ==0:
        print(d_)

07/23/20
07/14/20
07/14/20
07/13/20
07/01/20
06/30/20
06/29/20
06/25/20
06/18/20
06/07/20
06/04/20
06/01/20
05/27/20
05/06/20
04/24/20


### 9.rating

In [134]:
# stars_list = driver.find_elements_by_class_name('sr-only')
star_li = driver.find_elements_by_xpath('//span[@aria-label="Link to product page"]')
len(star_li)
for s in star_li[-15:]:
    stars = s.text
    stars = int(stars[:1])
    print(stars)

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


In [135]:
driver.get('https://www.bluenile.com/build-your-own-ring/petite-twist-diamond-engagement-ring-14k-white-gold_56003?yoReviewsPage=1')
reviews = driver.find_elements_by_xpath('//div[@class="yotpo-reviews yotpo-active"]/div')
reviews = reviews[3:]
item = {}
# for review in reviews:
#     print(review.text)

for review in reviews:
    question_list = review.find_elements_by_class_name('yotpo-user-field')
    row_dict = {}
    for question_ans in question_list:
        key = question_ans.find_element_by_xpath('./span[1]').text
        value = question_ans.find_element_by_xpath('./span[2]').text
        if not key:
            continue

        row_dict[key.replace(':', '')] = value
            
        user_name = review.find_element_by_xpath('.//span[@class="y-label yotpo-user-name yotpo-font-bold pull-left"]').text
        verified = review.find_element_by_xpath('.//span[@class="y-label yotpo-user-title yotpo-action-hover"]').text
        stars = review.find_element_by_xpath('.//span[@aria-label="Link to product page"]').text
        stars = int(stars[:1])
        date = review.find_element_by_xpath('.//span[@class="y-label yotpo-review-date"]').text
        review_text = review.find_element_by_xpath
            
        item['user_name'] = user_name
        item['verified'] = verified
        item['stars'] = stars
        item['date'] = date
        item['location'] = row_dict.get('Location')
        item['expert_consulted'] = row_dict.get('Blue Nile jewelry expert consulted')
        if row_dict.get('Recommendation'):
            item['recommended'] = row_dict.get('Recommendation')
        else:
            item['recommended'] = row_dict.get('Recommend')
        if row_dict.get('Why buy a Blue Nile Diamond'):
            item['why_buy'] = row_dict.get('Why buy a Blue Nile Diamond')
        else:
            item['why_buy'] = row_dict.get('Reason')
    print(item)    
#     csv_writer.writerow(item.values())


{'user_name': 'Noah S.', 'verified': 'Verified Buyer', 'stars': 5, 'date': '07/23/20', 'location': 'Palmyra, USA', 'expert_consulted': 'yes', 'recommended': 'Yes', 'why_buy': 'Quality'}
{'user_name': 'Ray W.', 'verified': 'Verified Buyer', 'stars': 5, 'date': '07/14/20', 'location': 'Fitchburg, USA', 'expert_consulted': 'yes', 'recommended': 'Yes', 'why_buy': 'Value'}
{'user_name': 'JONATHAN T.', 'verified': 'Verified Buyer', 'stars': 5, 'date': '07/14/20', 'location': 'SEVERANCE, USA', 'expert_consulted': 'yes', 'recommended': 'Yes', 'why_buy': 'Value'}
{'user_name': 'Zack C.', 'verified': 'Verified Buyer', 'stars': 5, 'date': '07/13/20', 'location': 'Parker, USA', 'expert_consulted': 'yes', 'recommended': 'Yes', 'why_buy': 'Value'}
{'user_name': 'Zack C.', 'verified': 'Verified Buyer', 'stars': 5, 'date': '07/13/20', 'location': 'Parker, USA', 'expert_consulted': 'yes', 'recommended': 'Yes', 'why_buy': 'Value'}
{'user_name': 'tim p.', 'verified': 'Verified Buyer', 'stars': 5, 'date':

In [80]:
q_a_dict

{'Recommendation': 'Yes',
 'Reason': 'Value',
 'Sweepstakes Opt-In': 'Yes',
 'Blue Nile jewelry expert consulted': 'no',
 'Location': 'Woodstock, USA',
 'Recommend': 'Yes',
 'Why buy a Blue Nile Diamond': 'Value'}

In [ ]:
recommend = q_a_dict.get('Recommend')
why_buy = q_a_dict.get('Why buy a Blue Nile Diamond')
location = q_a_dict.get('Location')

In [76]:
review_chuck = driver.find_elements_by_class_name("yotpo-header-element ")
for line,review in enumerate(review_chuck):
    content = review.text
    print(f'{line}:{content}')

0:
1:
2:
3:M
4:Michael H.
Verified Buyer
5 star rating
Blue Nile jewelry expert consulted:
Yes
CHURCHVILLE, NY
5:07/13/16
6:A
7:Anonymous
Verified Buyer
5 star rating
Cape Charles, VA
8:07/11/16
9:J
10:Jacob
Verified Buyer
5 star rating
Blue Nile jewelry expert consulted:
no
Ponca City, USA
11:07/08/16
12:A
13:Anonymous
Verified Buyer
5 star rating
Roswell, GA
14:06/08/16
15:A
16:Anonymous
Verified Buyer
4 star rating
Blue Nile jewelry expert consulted:
Yes
HOUSTON, TX
17:12/28/15
18:M
19:Marco
Verified Buyer
4 star rating
Mexico, Mexico
20:12/17/15
